In [ ]:
#Merge outcomes
#[x]Nonsurvivor = death or discharge to hospice 
#[]ventilator
#[]dialysis
#[]CPR
#[]trach/peg
#vasopressors 

In [ ]:
import os, re, math
import pandas as pd
import numpy as np
import time
from datetime import datetime

In [ ]:
%%time

notes = pd.read_csv('data-raw/mimic/NOTEEVENTS.csv', 
                    usecols=['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'CATEGORY', 'DESCRIPTION', 'CHARTTIME','CGID', 'TEXT'],
                    low_memory = False)
patients = pd.read_csv('data-raw/mimic/PATIENTS.csv',
                       usecols=['SUBJECT_ID', 'DOB', 'GENDER', 'EXPIRE_FLAG'],
                       low_memory = False)
admissions = pd.read_csv('data-raw/mimic/ADMISSIONS.csv', 
                         usecols=['SUBJECT_ID', 'HADM_ID', 'ADMITTIME', 'DISCHTIME', 
                                  'ADMISSION_TYPE', 'DISCHARGE_LOCATION', 'INSURANCE', 
                                  'MARITAL_STATUS', 'LANGUAGE', 'ETHNICITY', 'DIAGNOSIS'], low_memory = False)
diagnoses = pd.read_csv('data-raw/mimic/DIAGNOSES_ICD.csv', low_memory = False)
icu_stay = pd.read_csv('data-raw/mimic/ICUSTAYS.csv', low_memory = False)

# Created in 01_extract_outcomes.ipynb
chart_events = pd.read_csv('data/MIMIC_001_chartevents.csv', usecols=['SUBJECT_ID', 'HADM_ID', 'ITEMID', 
                                                                              'CHARTTIME', 'Mech_Vent', 'Trach'])
input_events = pd.read_csv('data/MIMIC_001_inputevents.csv', usecols=['SUBJECT_ID', 'HADM_ID', 'ITEMID', 
                                                                              'CHARTTIME', 'Pressors'])
all_event_id = pd.read_csv('data-raw/mimic/D_ITEMS.csv', usecols=['ITEMID', 'LABEL'])


In [ ]:

hd_chart = pd.read_csv('data/MIMIC_001_chartevents_hd.csv', usecols=['SUBJECT_ID', 'HADM_ID', 'ITEMID', 
                                                                              'CHARTTIME', 'Dialysis'])
hd_data = pd.read_csv('data/MIMIC_001_dataevents_hd.csv', usecols=['SUBJECT_ID', 'HADM_ID', 'ITEMID', 
                                                                              'CHARTTIME', 'Dialysis'])
hd_input = pd.read_csv('data/MIMIC_001_inputevents_hd.csv', usecols=['SUBJECT_ID', 'HADM_ID', 'ITEMID', 
                                                                              'CHARTTIME', 'Dialysis'])
hd_output = pd.read_csv('data/MIMIC_001_outputevents_hd.csv', usecols=['SUBJECT_ID', 'HADM_ID', 'ITEMID', 
                                                                              'CHARTTIME', 'Dialysis'])  
hd_proc = pd.read_csv('data/MIMIC_001_procevents_hd.csv',
                    usecols = ['SUBJECT_ID', 'HADM_ID', 'ITEMID', 'STARTTIME', 'Dialysis'])

In [ ]:
print("Total number of patients with notes:", len(notes.SUBJECT_ID.unique().tolist()))
print("Total number of patients:", len(patients.SUBJECT_ID.unique().tolist()))
print("Total number of patients in admissions:", len(admissions.SUBJECT_ID.unique().tolist()))
print("Total number of patients with ICD9:", len(diagnoses.SUBJECT_ID.unique().tolist()))
print("Total number of notewriters:", len(notes.HADM_ID.unique().tolist()))

In [ ]:
hd_proc['CHARTTIME']=hd_proc['STARTTIME']
hd_proc.pop('STARTTIME')
print('')

In [ ]:
hd0=pd.concat([hd_chart, hd_data]).drop_duplicates(['SUBJECT_ID', 'HADM_ID', 'ITEMID', 'CHARTTIME'])
hd1=pd.concat([hd0, hd_input]).drop_duplicates(['SUBJECT_ID', 'HADM_ID', 'ITEMID', 'CHARTTIME'])
hd2=pd.concat([hd1, hd_output]).drop_duplicates(['SUBJECT_ID', 'HADM_ID', 'ITEMID', 'CHARTTIME'])
hd_df=pd.concat([hd2, hd_input]).drop_duplicates(['SUBJECT_ID', 'HADM_ID', 'ITEMID', 'CHARTTIME'])

In [ ]:
len(hd_df.SUBJECT_ID.unique())

In [ ]:
len(input_events.SUBJECT_ID.unique())

In [ ]:
# Review distribution of the notes category 
notes.CATEGORY.value_counts()

In [ ]:
#Isolate columns of interest only
#patients_clean=patients[['SUBJECT_ID', 'DOB', 'GENDER', 'EXPIRE_FLAG']]
#notes_clean=notes[['SUBJECT_ID', 'HADM_ID', 'CATEGORY', 'DESCRIPTION', 'CHARTTIME','CGID', 'TEXT']]
#admissions_clean=admissions[['SUBJECT_ID', 'HADM_ID', 'ADMITTIME', 'DISCHTIME', 'ADMISSION_TYPE', 'DISCHARGE_LOCATION', 'INSURANCE', 'MARITAL_STATUS', 'LANGUAGE', 'ETHNICITY', 'DIAGNOSIS']]

In [ ]:
input_events.head(5)

In [ ]:
all_event_id

In [ ]:
all_event_id.LABEL.unique()

In [ ]:
#Merge patient and admission data
merge0=pd.merge(patients, admissions, on='SUBJECT_ID', how='left')

#get the age per patient
merge0['date_of_admission'] = pd.to_datetime(merge0['ADMITTIME']).dt.date
merge0['birthdate'] = pd.to_datetime(merge0['DOB']).dt.date
merge0['AGE'] = merge0.apply(lambda e: round((e['date_of_admission'] - e['birthdate']).days/365, 0), axis=1)
merge0=merge0.drop(['DOB', 'birthdate'], axis=1)

#Merge compiled patient data with notes 
merge1=pd.merge(merge0, notes, on=['SUBJECT_ID','HADM_ID'], how='left')

#Merge chart_events and ITEM ID LABELfor MV and Trach
merge2=pd.merge(chart_events, all_event_id, on='ITEMID', how='left')

#Merge input_events and ITEM ID LABEL for Pressors
merge3=pd.merge(input_events, all_event_id, on='ITEMID', how='left')

#Merge hd_df and ITEM ID LABEL for HD 
merge4=pd.merge(hd_df, all_event_id, on='ITEMID', how='left')


In [ ]:
merge4.head(5)

In [ ]:
merge2['CHARTTIME']=pd.to_datetime(merge2.CHARTTIME)
merge2['event_date']=merge2.CHARTTIME.dt.date
merge2=merge2.drop(['CHARTTIME', 'ITEMID', 'LABEL'], axis=1).drop_duplicates(keep='last')

In [ ]:
merge3['CHARTTIME']=pd.to_datetime(merge3.CHARTTIME)
merge3['event_date']=merge3.CHARTTIME.dt.date
merge3=merge3.drop(['CHARTTIME', 'ITEMID', 'LABEL'], axis=1).drop_duplicates(keep='last')

In [ ]:
merge4['CHARTTIME']=pd.to_datetime(merge4.CHARTTIME)
merge4['event_date']=merge4.CHARTTIME.dt.date
merge4=merge4.drop(['CHARTTIME', 'ITEMID', 'LABEL'], axis=1).drop_duplicates(keep='last')

In [ ]:
mv_days=merge2[merge2.Mech_Vent==1].groupby(['HADM_ID'])['event_date'].apply(lambda x: x.nunique())
mv_start=merge2[merge2.Mech_Vent==1].groupby(['HADM_ID'])['event_date'].min()
mv_end=merge2[merge2.Mech_Vent==1].groupby(['HADM_ID'])['event_date'].max()
mv_df=pd.concat([mv_days, mv_start, mv_end], axis=1).reset_index()
mv_df.columns=['HADM_ID', 'num_mv_days', 'mv_start_date','mv_end_date']

trach_days=merge2[merge2.Trach==1].groupby(['HADM_ID'])['event_date'].apply(lambda x: x.nunique())
trach_start=merge2[merge2.Trach==1].groupby(['HADM_ID'])['event_date'].min()
trach_end=merge2[merge2.Trach==1].groupby(['HADM_ID'])['event_date'].max()
trach_df=pd.concat([trach_days, trach_start, trach_end], axis=1).reset_index()
trach_df.columns=['HADM_ID', 'num_trach_days', 'trach_start_date','trach_end_date']

dialysis_days=merge4[merge4.Dialysis==1].groupby(['HADM_ID'])['event_date'].apply(lambda x: x.nunique())
dialysis_start=merge4[merge4.Dialysis==1].groupby(['HADM_ID'])['event_date'].min()
dialysis_end=merge4[merge4.Dialysis==1].groupby(['HADM_ID'])['event_date'].max()
dialysis_df=pd.concat([dialysis_days, dialysis_start, dialysis_end], axis=1).reset_index()
dialysis_df.columns=['HADM_ID', 'num_dialysis_days', 'dialysis_start_date','dialysis_end_date']

pressors_days=merge3[merge3.Pressors==1].groupby(['HADM_ID'])['event_date'].apply(lambda x: x.nunique())
pressors_start=merge3[merge3.Pressors==1].groupby(['HADM_ID'])['event_date'].min()
pressors_end=merge3[merge3.Pressors==1].groupby(['HADM_ID'])['event_date'].max()
pressors_df=pd.concat([pressors_days, pressors_start, pressors_end], axis=1).reset_index()
pressors_df.columns=['HADM_ID', 'num_pressor_days', 'pressors_start_date','pressors_end_date']


In [ ]:
print(len(mv_df.HADM_ID.unique().tolist()))
print(len(trach_df.HADM_ID.unique().tolist()))
print(len(dialysis_df.HADM_ID.unique().tolist()))
print(len(pressors_df.HADM_ID.unique().tolist()))

In [ ]:
ev0=pd.merge(mv_df, trach_df, on='HADM_ID', how='left')
ev1=pd.merge(ev0, dialysis_df, on='HADM_ID', how='left')
ev2=pd.merge(ev1, pressors_df, on='HADM_ID', how='left')
ev2=ev2.fillna({'num_intubated_days': 0, 'num_trach_days': 0, 'num_dialysis_days':0, 'num_pressor_days':0})

In [ ]:
#Merge merge1 with merge2
merge5=pd.merge(merge1, ev2, on=['HADM_ID'], how='left')

In [ ]:
merge5.shape

In [ ]:
#Only including the 8 notetype categories of interest 
merge6=merge5[(merge5.CATEGORY=='Discharge summary') | (merge5.CATEGORY=='Nursing') | 
                         (merge5.CATEGORY=='Physician ') | (merge5.CATEGORY=='General') | 
                        (merge5.CATEGORY=='Consult') | (merge5.CATEGORY=='Respiratory ') |
                        (merge5.CATEGORY=='Rehab Services') | (merge5.CATEGORY=='Nutrition')]

In [ ]:
#Important: This deletes note entries that are duplicates of each other or very similar due to minor text updates (i.e. same note type & CGID)
# CK: this may be a mistake.
all_patients=merge6.drop_duplicates(subset=['SUBJECT_ID','ADMITTIME', 'DIAGNOSIS', 'CATEGORY', 'DESCRIPTION', 'CGID'], keep='last')

In [ ]:
#Shows all the possible discharge locations 
all_patients.DISCHARGE_LOCATION.unique()

In [ ]:
#Death or Hospice as outcome
def mortality_outcome(string): 
    if string=='DEAD/EXPIRED': return 1
    elif string=='HOSPICE-HOME': return 1
    elif string=='HOSPICE-MEDICAL FACILITY': return 1
    else: return 0
all_patients['DC_TO_DEATH_HOSPICE']=all_patients.DISCHARGE_LOCATION.map(lambda x: mortality_outcome(x))

In [ ]:
#Cleaning of text
all_patients['TEXT']=all_patients.TEXT.map(lambda x: x.replace('\n', ' '))

In [ ]:
#LOS as outcome
all_patients['LENGTH_OF_STAY']=(pd.to_datetime(all_patients['DISCHTIME']).dt.date-pd.to_datetime(all_patients['ADMITTIME']).dt.date)

In [ ]:
#Final counts of notes in each category
all_patients.CATEGORY.value_counts()

In [ ]:
all_patients['days_until_mechvent']=all_patients.mv_start_date-all_patients.date_of_admission
all_patients['days_until_dialysis']=all_patients.dialysis_start_date-all_patients.date_of_admission
all_patients['days_until_pressors']=all_patients.pressors_start_date-all_patients.date_of_admission
all_patients['days_until_trach']=all_patients.trach_start_date-all_patients.date_of_admission

In [ ]:
all_patients.columns.tolist()

In [ ]:
pmh=pd.read_csv('data/MIMIC_001_Elixhauser.csv')
oasis=pd.read_csv('data/MIMIC_001_Oasis.csv')

In [ ]:
pmh.head(5)

In [ ]:
oasis.head(3)

In [ ]:
fin_merge0=pd.merge(all_patients, pmh, on=['SUBJECT_ID', 'HADM_ID'], how='left')
fin_merge1=pd.merge(fin_merge0, oasis, on=['SUBJECT_ID', 'HADM_ID'], how='left')

In [ ]:
print(merge1.columns.tolist())

In [ ]:
def oasis_score(x):
    pre_icu_score=0
    age_score=0
    gcs_score=0
    hr_score=0
    map_score=0
    rr_score=0
    temp_score=0
    uop_score=0
    mv_score=0
    elect_score=0
    
    pre_icu_los=x['pre_icu_los']
    if pre_icu_los < 1: pre_icu_score=0
    elif pre_icu_los < 13: pre_icu_score=1
    
    age=x['AGE']
    if age<24: age_score=0
    elif age <=53: age_score=3
    elif age <=77: age_score=6
    elif age<=89: age_score=9
    elif age >=90: age_score=7
    
    mingcs=x['GCS_min']
    if mingcs<=7: gcs_score=10
    elif mingcs<14: gcs_score=4
    elif mingcs==14: gcs_score=3 
        
    hrmax=x['HR_max']
    hrmin=x['HR_min']
    if hrmax>125: hr_score=6
    elif hrmin<33: hr_score=4
    elif ((hrmax>=107) & (hrmax<=125)): hr_score=3
    elif ((hrmax>=89) & (hrmax<=106)): hr_score=1
        
    mapmin=x['MAP_min']
    mapmax=x['MAP_max']
    if mapmin<20.65: map_score=4
    elif mapmin<51: map_score=3
    elif mapmax>143.44: map_score=3
    elif ((mapmin>=51) & (mapmin<61.33)): map_score=2
        
    rrmin=x['RR_min']
    rrmax=x['RR_max']
    if rrmin < 6: rr_score=10
    elif rrmax>44: rr_score=9
    elif rrmax>30: rr_score=6
    elif rrmax>22: rr_score=1
    elif rrmin<13: rr_score=1
        
    tempmax=x['temp_max']
    tempmin=x['temp_min']
    if tempmax>39.88: temp_score=6
    elif ((tempmin>=33.22) & (tempmin<35.93)): temp_score=4
    elif ((tempmax>=33.22) & (tempmax<=35.93)): temp_score=4
    elif tempmin<33.22: temp_score=3
    elif ((tempmin>35.93) & (tempmin<=36.36)): temp_score=2
    elif ((tempmax>=36.89) & (tempmax<=39.88)): temp_score=2
    
    uop=x['total_uop']
    if uop<671.09: uop_score=10
    elif uop>6896.8: uop_score=8
    elif ((uop>=671.09) & (uop<1426.99)): uop_score=5
    elif ((uop>1427.00) & (uop<=2544.14)): uop_score=1
    
    mv=x['Mech_Vent']
    if mv==1: mv_score=9
    
    elect=x['elective_adm']
    if elect!=1: elect_score=6
    
    return pre_icu_score + age_score + gcs_score + hr_score + map_score + rr_score + temp_score + uop_score + mv_score + elect_score

In [ ]:
fin_merge1['OASIS_score']=fin_merge1.apply(oasis_score, axis=1)

In [ ]:
fin_merge1['OASIS_prob']=fin_merge1.OASIS_score.map(lambda x: 1 / (1 + math.exp((-(-6.1746 + 0.1275*(x))))))

In [ ]:
fin_merge1.to_csv('data/MIMIC_001_merge_patients_4.20.csv', index=False)

In [ ]:
fin_merge1.reset_index(drop = True).to_feather('data/02_merge_patients.feather')